<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Обоснование" data-toc-modified-id="Обоснование-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обоснование</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

**Цель исследования:** необходимо защитить данные клиентов страховой компании «Хоть потоп». Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, обосновать корректность его работы. Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Этапы исследования:**
1. Загрузка данных;
2. Умножение матриц;
3. Создание алгоритма преобразования;
4. Проверка алгоритма.

## Загрузка данных

Импортируем необходимые для работы библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузим данные в переменную и изучим их.

In [2]:
data = pd.read_csv("/datasets/insurance.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(15)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


**Описание данных:**

* Признаки:

`Пол застрахованного клиента;`

`Возраст застрахованного клиента;` 

`Зарплата застрахованного клиента;` 

`Члены семьи` — количество членов семьи застрахованного клиента. 

* Целевой признак:

`Страховые выплаты` - количество страховых выплат за 5 лет. 

Проверим данные на наличие дубликатов.

In [6]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


### Вывод:

1. Названия столбцов оформлены в одном стиле.
2. Имеются столбцы с типами данных, которые можно было бы заменить на менее затратные в плане памяти.
3. В данных отсутствуют пропуски.
4. Были удалены явные дубликаты.

## Умножение матриц

Необходимо ответить на вопрос и обосновать решение:

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

a. Изменится. Приведите примеры матриц.

b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** b. 

### Обоснование

**Обозначения:**

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w*$ — вектор весов линейной регрессии матрицы полученной домножение исходной на обратимую матрицу(нулевой элемент равен сдвигу)

- $\hat{y}$ — предсказания по исходной матрице признаков

- $\hat{y}*$ — предсказания по матрице полученной домноженной на обратимую матрицу

**Основные свойства обратной матрицы:**

$$det(A) = 1/det(A) $$,
$$(AB)^{-1} = B^{-1}A^{-1}$$ 
для двух квадратных обратимых матриц $A$ и $B$, 
$$(A^T)^{-1} = ((A)^{-1})^T$$.
 

**Основные свойства единичной матрицы:**

$$AE = EA = A$$,
$$AA^{-1}=E$$.

**Основные свойства транспонированной матрицы:**

$$(AB)^{T} = B^{T}A^{T}$$ 

**Сочетательное свойство матриц, ассоциативность:**

$$A(BC) = (AB)C$$

**Предсказания:**

$$
a = Xw
$$

**Задача обучения:**

$$
w = \arg\min_w MSE(Xw, y)
$$

**Формула обучения:**

$$
w = (X^T X)^{-1} X^T y
$$

**Исходная матрица признаков:**

$$X$$


**Формула обучения:**

$$w = (X^T X)^{-1} X^T y$$

*Распишем формулу предсказания:*

$$a = X (X^T X)^{-1} X^T y$$
    
**Новая матрица признаков:**

$$X.dot(P)$$

**Предсказания:**

$$a* = XPw*$$

**Формула обучения:**

$$w* = ((XP)^T XP)^{-1} (XP)^T y$$

*Распишем формулу предсказания:*

$$a* = XP ((XP)^T XP)^{-1} (XP)^T y$$

*Раскроем скобки по свойству транспонированных матриц и добавим скобки, чтобы получить квадратную матрицу по свойству ассоциативности:*

$$a* = X P (P^T (X^T X) P)^{-1} P^T X^T y$$

*Раскроем скобки по свойству обратных матриц:*

$$a* = X P P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y$$

**По свойству единичной матрицы, все P в формуле сокращаются, а значит, умножение матрицы признаков на случайную обратимую матрицу не влияет на качество линейной регрессии**

## Алгоритм преобразования

Необходимо предложить алгоритм преобразования данных для решения задачи и обосновать, почему качество линейной регрессии не поменяется.

**Алгоритм**

Необходимо умножить матрицу признаков на обратимую матрицу.

**Обоснование**

Умножив матрицу признаков на случайную обратимую матрицу, мы зашифруем истинные значения признаков. Из за того, что матрица для умножения случайна, результаты предсказания до умножения и после идентичны (доказано в п.2,1), а размеры самих матриц очень велики, то случайную матрицу, которая выступает в роли ключа, угадать практичечки невозможно. 

Из одинаковых предсказаний так же следует то, что метрики не изменятся, а следовательно и качество линейной регрессии.

## Проверка алгоритма

Выделим признаки и целевой признак.

In [7]:
X = data.drop(labels = "Страховые выплаты",axis = 1)
y = data["Страховые выплаты"]

In [8]:
print(X.shape,y.shape)

(4847, 4) (4847,)


Добавим в начало матрицы столбец, заполненный единицами, соответствующий $w0$.

In [9]:
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X.shape

(4847, 5)

Для проверки алгоритма нам необходимо сгенерировать случайную обратимую матрицу.

In [10]:
def invertible_matrix():
    
    len_X = X.shape[1]
    
    r = np.random.RandomState(777)
    
    P = r.rand(len_X, len_X)
    
    try: 
        
        np.linalg.inv(P)
        return P
        
    except:
        
        r = np.random.RandomState(randint(0 , 100))
        P = r.rand(len_X, len_X)
        
        try: 
        
            np.linalg.inv(P)
            return P
        
        except:
        
            r = np.random.RandomState(randint(0 , 100))
            P = r.rand(len_X, len_X)

            return P

In [11]:
P = invertible_matrix()
P

array([[0.15266373, 0.30235661, 0.06203641, 0.45986034, 0.83525338],
       [0.92699705, 0.72698898, 0.76849622, 0.26920507, 0.64402929],
       [0.09337326, 0.07968589, 0.58961375, 0.34334054, 0.98887615],
       [0.62647321, 0.68177928, 0.55225681, 0.26886006, 0.37325939],
       [0.2229281 , 0.1864426 , 0.39064809, 0.19316241, 0.61091093]])

In [12]:
XP = X @ P

Теперь найдем предсказания для исходных данных, и для зашифрованных.

In [13]:
W = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
WP = (np.linalg.inv(XP.T.dot(XP))).dot(XP.T).dot(y)

In [14]:
A = X.dot(W)
AP = XP.dot(WP)

In [15]:
print (A)
print (AP)

[ 0.51932881  0.69230097  0.09412742 ... -0.26164511 -0.19394186
  0.05044763]
[ 0.51933632  0.69230138  0.09412943 ... -0.26164782 -0.19394085
  0.05045453]


In [16]:
print("R2_score для исходной матрицы = ",r2_score(y,A))
print("R2_score для зашифрованной матрицы = ",r2_score(y,AP))

R2_score для исходной матрицы =  0.4302010044852068
R2_score для зашифрованной матрицы =  0.43020100435102515


Предсказания отличаются друг от друга незначительно (от 1 * 10^(-5)).

Теперь проверим это на линейной регрессии из sklearn.

In [17]:
model = LinearRegression(fit_intercept=False)
model.fit(X,y)
predict = model.predict(X)
print("R2_score для исходной матрицы = ",r2_score(y,predict))

R2_score для исходной матрицы =  0.4302010044852068


In [18]:
model = LinearRegression(fit_intercept=False)
model.fit(XP,y)
predict = model.predict(XP)
print("R2_score для зашифрованной матрицы = ",r2_score(y,predict))

R2_score для зашифрованной матрицы =  0.430201004485092


Метрики также отличаются друг от друга минимально.